### Stokes problem
We consider the Stokes problem on the unit square $\Omega = (0,1)^2$ with non-homogeneous Dirichlet boundary conditions ($u=(1,0)$) only on the top.

Find $\mathbf{u} \in \mathbf{V}$, $p \in Q$, $\lambda \in \mathbb{R}$ so that
\begin{align}
\int_{\Omega} \nu \nabla \mathbf{u} : \nabla \mathbf{v} &- \int_{\Omega} \operatorname{div}(\mathbf{v}) p & &= \int \mathbf{f}  \cdot \mathbf{v}  && \forall \mathbf{v} \in \mathbf{V}, \\ 
- \int_{\Omega} \operatorname{div}(\mathbf{u}) q & & 
+ \int_{\Omega} \lambda q
&= 0 && \forall q \in Q, \\
& \int_{\Omega} \mu p & &= 0 && \forall \mu \in \mathbb{R}.
\end{align}


In [ ]:
from ngsolve.solvers import *
from ngsolve.meshes import *
from ngsolve import *
from draw import Draw
from netgen.geom2d import unit_square

In [ ]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.1))

In [ ]:
Vh = VectorH1(mesh,order=2,dirichlet="bottom|right|top|left")
Qh = H1(mesh,order=1); 
Nh = NumberSpace(mesh); 
Wh = FESpace([Vh,Qh,Nh])

In [ ]:
(u,p,lam), (v,q,mu) = Wh.TnT()
a = BilinearForm(Wh)
a += (InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p-lam*q-mu*p)*dx

In [ ]:
gfu = GridFunction(Wh)
gfu.components[0].Set(CoefficientFunction((x*(1-x),0)),
                      definedon=mesh.Boundaries("top"))

In [ ]:
a.Assemble()
f = LinearForm(Wh)
f.Assemble()

f.vec.data -= a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(Wh.FreeDofs()) * f.vec
print(Norm(gfu.vec))

In [ ]:
Draw(0.05*gfu.components[1],mesh,"p")

In [ ]:
Draw(gfu.components[0],mesh,"u")